# Biodiversity in National Parks

In this project, we will be examine data gathered from US National Parks. The goal is to find which parks contain the most diversity within.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Import Data

I have imported the required CSVs for the project then we will start to examine them in the next section.

In [2]:
species_data = pd.read_csv("species_info.csv")
observation_data = pd.read_csv("observations.csv")

## Examine Species Data

In this section, I have added lines to explore the data and to know what is contained in the Species Data.

In [3]:
print("Species")
print(species_data.info())
print(species_data.describe())
print(species_data.head())

Species
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5824 entries, 0 to 5823
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   category             5824 non-null   object
 1   scientific_name      5824 non-null   object
 2   common_names         5824 non-null   object
 3   conservation_status  191 non-null    object
dtypes: object(4)
memory usage: 182.1+ KB
None
              category    scientific_name        common_names  \
count             5824               5824                5824   
unique               7               5541                5504   
top     Vascular Plant  Castor canadensis  Brachythecium Moss   
freq              4470                  3                   7   

       conservation_status  
count                  191  
unique                   4  
top     Species of Concern  
freq                   161  
  category                scientific_name  \
0   Mammal  Clethrionomys gap

## Examine Species Data

In this section, I have added lines to explore the data and to know what is contained in the Observation Data.

In [9]:
print("Observation")
print(observation_data.info())
print(observation_data.describe())
print(observation_data.head())


Observation
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23296 entries, 0 to 23295
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   scientific_name  23296 non-null  object
 1   park_name        23296 non-null  object
 2   observations     23296 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 546.1+ KB
None
       observations
count  23296.000000
mean     142.287904
std       69.890532
min        9.000000
25%       86.000000
50%      124.000000
75%      195.000000
max      321.000000
            scientific_name                            park_name  observations
0        Vicia benghalensis  Great Smoky Mountains National Park            68
1            Neovison vison  Great Smoky Mountains National Park            77
2         Prunus subcordata               Yosemite National Park           138
3      Abutilon theophrasti                  Bryce National Park            84
4  Githopsis speculario

## Connection the Tables

